# 🤗 End-to-end distilabel example with Inference Endpoints and Notus

In [1]:
import os
import time
from typing import Dict

import argilla as rg

from distilabel.llm import InferenceEndpointsLLM
from distilabel.pipeline import Pipeline, pipeline
from distilabel.tasks import Llama2TextGenerationTask, SelfInstructTask, Prompt

from datasets import Dataset

/Users/ignacio/.local/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
os.environ["HF_TOKEN"] = "hf_"
os.environ["OPENAI_API_KEY"] = "sk-"
os.environ["ARGILLA_API_URL"] = "https://argilla-ultrafeedback-curator.hf.space"
os.environ["ARGILLA_API_KEY"] = "admin.apikey"

## Setting up an inference endpoint with Notus

To kickstart this tutorial, let's see how to set up and endpoint for our Notus model. A HuggingFace endpoint is a service provided by HuggingFace that allows you to deploy and host your machine learning models for inference. This way, we'll have faster inference times, as these models will not run in our personal machines, but in HuggingFace servers. The endpoint of choice has a [Notus 7B instance](https://ui.endpoints.huggingface.co/argilla/endpoints/aws-notus-7b-v1-4052) running.

Let's see a quick example of how to use an inference endpoint. We have prepared an easy `Llama2QuestionAnsweringTask` to ask question to the model, in a very similar way as we talk with the LLMs using chatbots.

In [3]:
class Llama2QuestionAnsweringTask(Llama2TextGenerationTask):
    def generate_prompt(self, question: str) -> str:
        return Prompt(
            system_prompt=self.system_prompt,
            formatted_prompt=question,
        ).format_as("llama2")  # type: ignore

    def parse_output(self, output: str) -> Dict[str, str]:
        return {"answer": output.strip()}

    def input_args_names(self) -> list[str]:
        return ["question"]

    def output_args_names(self) -> list[str]:
        return ["answer"]

Once this class is ready, we have to instantiate an `InferenceEndpointsLLM` object, and pass as parameters the HF Inference Endpoint name and the HF namespace. One very convenient way to do so is through environment variables.

In [4]:
os.environ["HF_INFERENCE_ENDPOINT_NAME"] = "aws-notus-7b-v1-4052"
os.environ["HF_NAMESPACE"] = "argilla"

A HuggingFace Token is also required to use HuggingFace's services.

In [5]:
llm = InferenceEndpointsLLM(
    endpoint_name=os.getenv("HF_INFERENCE_ENDPOINT_NAME"),  # type: ignore
    endpoint_namespace=os.getenv("HF_NAMESPACE"),  # type: ignore
    token=os.getenv("HF_TOKEN") or None,
    task=Llama2QuestionAnsweringTask(),
)

The `llm` is an object of the `InferenceEndpointsLLM` class, and through it we can start generating answers to question using the `llm.generate()` method.

In [6]:
generation = llm.generate([{"question": "What's the capital of Spain?"}])
generation[0][0]["parsed_output"]["answer"]

"The capital of Spain is Madrid. It is the largest city in Spain and the third-largest city in the European Union. Madrid is known for its rich history, art, and culture, and is home to many famous landmarks, such as the Prado Museum, the Royal Palace of Madrid, and the Retiro Park. Madrid is also a major economic and financial center in Europe, and is home to many international companies and organizations. The city is known for its vibrant nightlife, delicious cuisine, and friendly people. Madrid is a great destination for travelers looking to experience the best of Spain's culture"

The endpoint is working! We now can do inference through the Inference Endpoint.

## Generating instructions with SelfInstructTask

With out Inference Endpoint up and running, we should be able to generate instructions with distilabel. These instructions, made by the LLM through our endpoint, will form an instruction dataset.

Firstly, let's provide some topics:

In [7]:
instructions_topics = [
    "Fantastic Stories",
    "Historic Novels",
    "Sci-fi Stories",
    "Fictional Characters",
    "Superheroes",
    "Superpowers"
]

In [8]:
instructions_dataset = Dataset.from_dict({
    "input": instructions_topics
})

As you can see, our topics are related to novels, Fantasy, Sci-fi and cool stories! Picture this: a LLM capable of creating characters for a fictional universe, with backstories, cool names and amazing superpowers. These topics will be used to generate instructions for the LLMs to run, as the are grouped in a `Dataset` class.

Now, for our generator to work, we need to create an Instruction Task, so the endpoint know how to treate these topics as inputs to generate the desired instructions. Using the `SelfInstructTask` class, which is based on a framework that uses the model's own generations to create a large collection of instructional data, we can generate instructions given the topics and a brief description of the desired behaviour.

In [9]:
instructions_task = SelfInstructTask(
    application_description="A assistant that creates fictional characters and their backstories."
)

Let's now define a generator, passing the `SelfInstructTask` object, and create a `Pipeline` object.

In [10]:
instructions_generator = InferenceEndpointsLLM(
    endpoint_name=os.getenv("HF_INFERENCE_ENDPOINT_NAME"),  # type: ignore
    endpoint_namespace=os.getenv("HF_NAMESPACE"),  # type: ignore
    token=os.getenv("HF_TOKEN") or None,
    task=instructions_task,
)

instructions_pipeline = Pipeline(
    generator=instructions_generator
)

Our pipeline is ready to be used to generate instructions. Let's do it!

In [11]:
generated_instructions = instructions_pipeline.generate(dataset=instructions_dataset, num_generations=4, batch_size=1)

/Users/ignacio/.local/lib/python3.11/site-packages/rich/live.py:231: UserWarning: install "ipywidgets" for Jupyter 
support
  warnings.warn('install "ipywidgets" for Jupyter support')

[12/07/23 11:56:11] INFO     INFO:distilabel:Processing batch 1 of 6...                             ]8;id=924569;file:///Users/ignacio/Documents/recognai/distilabel/src/distilabel/pipeline.py\pipeline.py]8;;\:]8;id=868035;file:///Users/ignacio/Documents/recognai/distilabel/src/distilabel/pipeline.py#525\525]8;;\

                    INFO     INFO:distilabel:Calling generator for batch 1...                       ]8;id=913746;file:///Users/ignacio/Documents/recognai/distilabel/src/distilabel/pipeline.py\pipeline.py]8;;\:]8;id=492282;file:///Users/ignacio/Documents/recognai/distilabel/src/distilabel/pipeline.py#529\529]8;;\

/Users/ignacio/Documents/recognai/distilabel/src/distilabel/llm/huggingface/inference_endpoints.py:177: 
UserWarning: No `prompt_format` has been specified and no `default_format` is set, so the prompt will be 
concatenated with a line-break and no specific formatting by default.
  prompts = self._generate_prompts(

[12/07/23 11:56:29] INFO     INFO:distilabel:Processing batch 2 of 6...                             ]8;id=305170;file:///Users/ignacio/Documents/recognai/distilabel/src/distilabel/pipeline.py\pipeline.py]8;;\:]8;id=736928;file:///Users/ignacio/Documents/recognai/distilabel/src/distilabel/pipeline.py#525\525]8;;\

...

[12/07/23 11:57:23] INFO     INFO:distilabel:Processing batch 5 of 6...                             ]8;id=161106;file:///Users/ignacio/Documents/recognai/distilabel/src/distilabel/pipeline.py\pipeline.py]8;;\:]8;id=442575;file:///Users/ignacio/Documents/recognai/distilabel/src/distilabel/pipeline.py#525\525]8;;\

                    INFO     INFO:distilabel:Calling generator for batch 5...                       ]8;id=949752;file:///Users/ignacio/Documents/recognai/distilabel/src/distilabel/pipeline.py\pipeline.py]8;;\:]8;id=770225;file:///Users/ignacio/Documents/recognai/distilabel/src/distilabel/pipeline.py#529\529]8;;\

[12/07/23 11:57:40] INFO     INFO:distilabel:Processing batch 6 of 6...                             ]8;id=290122;file:///Users/ignacio/Documents/recognai/distilabel/src/distilabel/pipeline.py\pipeline.py]8;;\:]8;id=660699;file:///Users/ignacio/Documents/recognai/distilabel/src/distilabel/pipeline.py#525\525]8;;\

                    INFO     INFO:distilabel:Calling generator for batch 6...                       ]8;id=607709;file:///Users/ignacio/Documents/recognai/distilabel/src/distilabel/pipeline.py\pipeline.py]8;;\:]8;id=383039;file:///Users/ignacio/Documents/recognai/distilabel/src/distilabel/pipeline.py#529\529]8;;\

Map: 100%|██████████| 6/6 [00:00<00:00, 865.43 examples/s]


Our pipeline has succesfully generated instructions given the topics and the behaviour passed as input. Let's gather all those instructions and see how the look.

In [12]:
instructions = []
for generations in generated_instructions["generations"]:
    for generation in generations:
        instructions.extend(generation)

print(f"Number of generated instructions: {len(instructions)}")

for instruction in instructions[:20]:
    print(instruction)

Number of generated instructions: 220
1. What is the origin story of the sorceress who controls the elements?
2. How did the half-dragon, half-human hybrid acquire their powers?
3. What led the cursed prince to seek redemption?
4. Describe the journey of the orphan who discovered their magical abilities.
5. How did the shape-shifting thief acquire their skills?
6. What motivated the mermaid to leave the ocean and explore the human world?
7. What was the inspiration behind the creation of the time-traveling wizard?
8. How did the
1. What is the origin story of the sorceress who controls the elements?
2. How did the half-dragon, half-human hybrid acquire their powers?
3. What led the cursed prince to seek redemption?
4. Describe the journey of the orphan who discovered their magical abilities.
5. How did the shape-shifting thief acquire their skills?
6. What motivated the mermaid to leave the ocean and explore the human world?
7. What was the inspiration behind the creation of the time-t

These instruction are really usefull in our story-making task, as we can start building a fictional world by just answering them.

## Generate a Preference Dataset using an Ultrafeedback text quality task.

Another possibility with Distilabel is to create a Preference Dataset through an Ultrafeedback text quality task. It's a type of task used in NLP to evaluate the quality of text generated. Our goal is to provide detailed feedback on the quality of the generated text, beyond just a binary label. 

Our `pipeline()` method allows us to create a `Pipeline` instance with the provided LLMs for a given task, which is useful whenever you want to use a pre-defined or custom `Pipeline` for a given task. We will specify our task and subtask, the generator we want to use (in this case, one based in a Llama2 Text Generator Task) and our OpenAI API key.

In [13]:
preference_pipeline = pipeline(
    "preference",
    "text-quality",
    generator=InferenceEndpointsLLM(
        endpoint_name=os.getenv("HF_INFERENCE_ENDPOINT_NAME"),  # type: ignore
        endpoint_namespace=os.getenv("HF_NAMESPACE", None),
        task=Llama2TextGenerationTask(),
        max_new_tokens=256,
        num_threads=2,
        temperature=0.3,
    ),
    max_new_tokens=256,
    num_threads=2,
    openai_api_key=os.getenv("OPENAI_API_KEY"),
    temperature=0.0,
)

[12/07/23 11:57:59] INFO     INFO:distilabel:Since no `labeller` was provided, `OpenAILLM` will be  ]8;id=451873;file:///Users/ignacio/Documents/recognai/distilabel/src/distilabel/pipeline.py\pipeline.py]8;;\:]8;id=978060;file:///Users/ignacio/Documents/recognai/distilabel/src/distilabel/pipeline.py#652\652]8;;\
                             used as the default labeller with `UltraFeedback`.                                    

Now, let's build a dataset by using the pipeline we just created, and the topics from which our instructions were generated. They are still valid, as we want to create a preference dataset still focus on writing characters and stories.

In [14]:
preference_dataset = preference_pipeline.generate(
    instructions_dataset,  # type: ignore
    num_generations=2,
    batch_size=1,
    enable_checkpoints=True,
    display_progress_bar=True,
)

/Users/ignacio/.local/lib/python3.11/site-packages/rich/live.py:231: UserWarning: install "ipywidgets" for Jupyter 
support
  warnings.warn('install "ipywidgets" for Jupyter support')

[12/07/23 11:58:00] INFO     INFO:distilabel:Processing batch 1 of 6...                             ]8;id=355955;file:///Users/ignacio/Documents/recognai/distilabel/src/distilabel/pipeline.py\pipeline.py]8;;\:]8;id=315066;file:///Users/ignacio/Documents/recognai/distilabel/src/distilabel/pipeline.py#525\525]8;;\

                    INFO     INFO:distilabel:Calling generator for batch 1...                       ]8;id=233254;file:///Users/ignacio/Documents/recognai/distilabel/src/distilabel/pipeline.py\pipeline.py]8;;\:]8;id=399760;file:///Users/ignacio/Documents/recognai/distilabel/src/distilabel/pipeline.py#529\529]8;;\

[12/07/23 11:58:18] INFO     INFO:distilabel:Calling labeller for batch 1...                        ]8;id=395085;file:///Users/ignacio/Documents/recognai/distilabel/src/distilabel/pipeline.py\pipeline.py]8;;\:]8;id=946214;file:///Users/ignacio/Documents/recognai/distilabel/src/distilabel/pipeline.py#552\552]8;;\

                    INFO     INFO:distilabel:Processing batch 2 of 6...                             ]8;id=617615;file:///Users/ignacio/Documents/recognai/distilabel/src/distilabel/pipeline.py\pipeline.py]8;;\:]8;id=390528;file:///Users/ignacio/Documents/recognai/distilabel/src/distilabel/pipeline.py#525\525]8;;\

...

[12/07/23 11:59:27] INFO     INFO:distilabel:Calling labeller for batch 5...                        ]8;id=587116;file:///Users/ignacio/Documents/recognai/distilabel/src/distilabel/pipeline.py\pipeline.py]8;;\:]8;id=151291;file:///Users/ignacio/Documents/recognai/distilabel/src/distilabel/pipeline.py#552\552]8;;\

                    INFO     INFO:distilabel:Processing batch 6 of 6...                             ]8;id=759928;file:///Users/ignacio/Documents/recognai/distilabel/src/distilabel/pipeline.py\pipeline.py]8;;\:]8;id=518910;file:///Users/ignacio/Documents/recognai/distilabel/src/distilabel/pipeline.py#525\525]8;;\

                    INFO     INFO:distilabel:Calling generator for batch 6...                       ]8;id=547874;file:///Users/ignacio/Documents/recognai/distilabel/src/distilabel/pipeline.py\pipeline.py]8;;\:]8;id=525161;file:///Users/ignacio/Documents/recognai/distilabel/src/distilabel/pipeline.py#529\529]8;;\

[12/07/23 11:59:45] INFO     INFO:distilabel:Calling labeller for batch 6...                        ]8;id=300038;file:///Users/ignacio/Documents/recognai/distilabel/src/distilabel/pipeline.py\pipeline.py]8;;\:]8;id=951514;file:///Users/ignacio/Documents/recognai/distilabel/src/distilabel/pipeline.py#552\552]8;;\

Map: 100%|██████████| 6/6 [00:00<00:00, 590.32 examples/s]


Let's take a look at an instance of the preference dataset

In [15]:
preference_dataset[0]

{'input': 'Fantastic Stories',
 'generation_model': 'argilla/notus-7b-v1',
 'generation_prompt': "<s>[INST] <<SYS>>\nYou are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe. Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature.\nIf a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information.<</SYS>>\n\nFantastic Stories [/INST]",
 'raw_generation_responses': ['\n\n1. What is the origin of the name "Fantastic Stories"?\n\nAnswer: The name "Fantastic Stories" was chosen by the publisher, Hugo Gernsback, for his science fiction magazine that was first published in 1933. Gernsback wanted a name that would convey the idea of imaginative and fantastic stories, and h

## Setting up an Argilla HF Space to upload the resulting dataset.

In [16]:
rg.init(
    api_url=os.getenv("ARGILLA_API_URL"), api_key=os.getenv("ARGILLA_API_KEY")
)

/Users/ignacio/Documents/recognai/argilla/src/argilla/client/client.py:165: UserWarning: No workspace configuration was detected. To work with Argilla datasets, specify a valid workspace name on `rg.init` or set it up through the `rg.set_workspace` function.
  warnings.warn(
/Users/ignacio/Documents/recognai/argilla/src/argilla/client/client.py:182: UserWarning: You're connecting to Argilla Server 1.19.0 using a different client version (1.20.0).
This may lead to potential compatibility issues during your experience.
To ensure a seamless and optimized connection, we highly recommend aligning your client version with the server version.
  warnings.warn(


In [17]:
# Uploading the Preference Dataset
preference_rg_dataset = preference_dataset.to_argilla()
preference_rg_dataset.push_to_argilla(name=f"notus_ultrafeedback_preference", workspace="admin")

[12/07/23 11:59:56] INFO     INFO:argilla.client.feedback.dataset.local.mixins:✓ Dataset succesfully  ]8;id=587870;file:///Users/ignacio/Documents/recognai/argilla/src/argilla/client/feedback/dataset/local/mixins.py\mixins.py]8;;\:]8;id=480983;file:///Users/ignacio/Documents/recognai/argilla/src/argilla/client/feedback/dataset/local/mixins.py#281\281]8;;\
                             pushed to Argilla                                                                     

                    INFO     INFO:argilla.client.feedback.dataset.local.mixins:RemoteFeedbackDataset( ]8;id=722722;file:///Users/ignacio/Documents/recognai/argilla/src/argilla/client/feedback/dataset/local/mixins.py\mixins.py]8;;\:]8;id=934426;file:///Users/ignacio/Documents/recognai/argilla/src/argilla/client/feedback/dataset/local/mixins.py#282\282]8;;\
                                id=9d9e0524-e410-4998-a48a-d2d16e4fe486                                            
                                name=notus_ultrafeedback_preference                                                
                                workspace=Workspace(id=1debb02b-ca36-4807-86f9-c493b7dbc30e,                       
                             name=admin, inserted_at=2023-11-13 18:32:30.307392,                                   
                             updated_at=2023-11-13 18:32:30.307392)                                                
                                url=https://argilla-ultrafeedback-curator.hf.space/dataset/9d9e0524-e              
                             410-4998-a48a-d2d16e4fe486/annotation-mode                                            
                                fields=[RemoteTextField(id=UUID('74fa2da0-df49-4cf4-a56b-44b3acce1f0b              
                             '), client=None, name='input', title='input', required=True,                          
                             type='text', use_markdown=False),                                                     
                             RemoteTextField(id=UUID('b16c57d7-3457-486a-ad1a-6e3eac337213'),                      
                             client=None, name='generations-1', title='generations-1', required=True,              
                             type='text', use_markdown=False),                                                     
                             RemoteTextField(id=UUID('df8a09a4-50b2-4dc4-af87-2218b73989a0'),                      
                             client=None, name='generations-2', title='generations-2', required=True,              
                             type='text', use_markdown=False)]                                                     
                                questions=[RemoteRatingQuestion(id=UUID('870e3091-8b07-4658-bfcd-0fc2              
                             883617dd'), client=None, name='generations-1-rating', title="What's the               
                             rating for generations-1?", description=None, required=True,                          
                             type='rating', values=[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]),                               
                             RemoteRatingQuestion(id=UUID('61580dee-4a28-48aa-ad4a-0e305673a27e'),                 
                             client=None, name='generations-2-rating', title="What's the rating for                
                             generations-2?", description=None, required=True, type='rating',                      
                             values=[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]),                                              
                             RemoteTextQuestion(id=UUID('295e81f7-725e-4729-83e2-7a6a2b739c57'),                   
                             client=None, name='rating-rationale', title="What's the rationale behind              
                             each rating?", description=None, required=True, type='text',                          
                             use_markdown=False)]                                                                  
                                guidelines=None                                                                    
                                metadata_properties=[RemoteIntegerMetadataProperty(id=UUID('7be48d7d-              
                             bb2a-4a74-8df4-8f7541834d77'), client=<httpx.Client object at                         
                             0x175b99f

RemoteFeedbackDataset(
   id=9d9e0524-e410-4998-a48a-d2d16e4fe486
   name=notus_ultrafeedback_preference
   workspace=Workspace(id=1debb02b-ca36-4807-86f9-c493b7dbc30e, name=admin, inserted_at=2023-11-13 18:32:30.307392, updated_at=2023-11-13 18:32:30.307392)
   url=https://argilla-ultrafeedback-curator.hf.space/dataset/9d9e0524-e410-4998-a48a-d2d16e4fe486/annotation-mode
   fields=[RemoteTextField(id=UUID('74fa2da0-df49-4cf4-a56b-44b3acce1f0b'), client=None, name='input', title='input', required=True, type='text', use_markdown=False), RemoteTextField(id=UUID('b16c57d7-3457-486a-ad1a-6e3eac337213'), client=None, name='generations-1', title='generations-1', required=True, type='text', use_markdown=False), RemoteTextField(id=UUID('df8a09a4-50b2-4dc4-af87-2218b73989a0'), client=None, name='generations-2', title='generations-2', required=True, type='text', use_markdown=False)]
   questions=[RemoteRatingQuestion(id=UUID('870e3091-8b07-4658-bfcd-0fc2883617dd'), client=None, name='generations